In [2]:
import pandas as pd
import folium

providers_path = r"C:\CTSHack\Dashboard\insurers.csv"
providers_df = pd.read_csv(providers_path)

patients_path = r"C:\CTSHack\Dashboard\updated_patients.csv"
patients_df = pd.read_csv(patients_path)

providers_df.columns = providers_df.columns.str.strip().str.replace("\t", "", regex=True)

required_cols = {"State", "City/Town", "Latitude", "Longitude", 
                 "Provider First Name", "pri_spec", "Facility Name"}
missing = required_cols - set(providers_df.columns)
if missing:
    raise ValueError(f"Missing required columns: {missing}")

providers_df["Latitude"] = pd.to_numeric(providers_df["Latitude"], errors="coerce")
providers_df["Longitude"] = pd.to_numeric(providers_df["Longitude"], errors="coerce")
providers_df = providers_df.dropna(subset=["Latitude", "Longitude"])

def assess_capacity(
    state: str,
    capacity_per_provider: int = 15,
):
    """
    state: US state to evaluate
    capacity_per_provider: max patients per provider
    """

    providers = providers_df[providers_df["State"].str.strip().str.casefold() == state.strip().casefold()]
    provider_count = len(providers)


    patients = patients_df[patients_df["State"].str.strip().str.casefold() == state.strip().casefold()]
    patient_count = len(patients)


    total_capacity = provider_count * capacity_per_provider
    adequacy = "Adequate" if patient_count <= total_capacity else "Inadequate"

    print(f"State: {state}")
    print(f"Providers found: {provider_count}")
    print(f"Patients found: {patient_count}")
    print(f"Capacity per provider: {capacity_per_provider}")
    print(f"Total provider capacity: {total_capacity}")
    print(f"Status: {adequacy}")

    if provider_count == 0:
        return adequacy, None


    center_lat, center_lon = providers["Latitude"].mean(), providers["Longitude"].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=6)


    for _, r in providers.iterrows():
        popup = (
            f"{r.get('Provider First Name','N/A')} "
            f"({r.get('pri_spec','N/A')})<br>"
            f"{r.get('Facility Name','N/A')}<br>"
            f"{r.get('City/Town','N/A')}, {r.get('State','N/A')}"
        )
        folium.Marker([r["Latitude"], r["Longitude"]], popup=popup, icon=folium.Icon(color="blue", icon="plus")).add_to(m)

    return adequacy, m



state_name = input("Enter the state : ")
adequacy, provider_map = assess_capacity(state_name, capacity_per_provider=15)

if provider_map is not None:
    filename = "map.html"
    provider_map.save(filename)
    print(f"✅ Map saved to {filename} — open it in your browser")


State: alaska
Providers found: 2
Patients found: 104
Capacity per provider: 15
Total provider capacity: 30
Status: Inadequate
✅ Map saved to map.html — open it in your browser
